In [37]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
import os
import pickle
import networkx as nx
import pandas as pd

sys.path.append(os.path.abspath(".."))

from kg.search_methods import direct_edge_search, neighbor_search, light_rag_search, adaptive_graph_rag_search, rq_rag_search

project_root = os.path.abspath("..")
graph_path = os.path.join(project_root, "data", "processed", "graph.gpickle")
interaction_info_path = os.path.join(project_root, "data", "raw", "Interaction_information.csv")
drug_info_path = os.path.join(project_root, "data", "raw", "Approved_drug_Information.txt")

with open(graph_path, "rb") as f:
    G = pickle.load(f)

interaction_info_df = pd.read_csv(interaction_info_path)
interaction_mapping = pd.Series(interaction_info_df.Description.values, index=interaction_info_df["Interaction type"]).to_dict()

drug_info_df = pd.read_csv(drug_info_path, sep="\t", header=None)
drug_info_df.columns = ["DrugBank_ID", "Drug_Name"] + [f"col_{i}" for i in range(len(drug_info_df.columns) - 2)]
name_to_id = pd.Series(drug_info_df.DrugBank_ID.values, index=drug_info_df.Drug_Name).to_dict()


In [3]:
drug1_id = "DB00682"  # Warfarin
drug2_id = "DB00945"  # Aspirin

if drug1_id not in G.nodes:
    raise ValueError(f"Drug ID {drug1_id} not in Knowledge Graph.")
if drug2_id not in G.nodes:
    raise ValueError(f"Drug ID {drug2_id} not in Knowledge Graph.")

found, description = direct_edge_search(G, drug1_id, drug2_id, interaction_mapping)
print("[Direct Edge Search]", "Found" if found else "Not Found", "|", description)

found, description = neighbor_search(G, drug1_id, drug2_id, interaction_mapping)
print("[Neighbor Search]", "Found" if found else "Not Found", "|", description)

found, description = light_rag_search(G, drug1_id, drug2_id, interaction_mapping)
print("[LightRAG Search]", "Found" if found else "Not Found", "|", description)

found, description = adaptive_graph_rag_search(G, drug1_id, drug2_id, interaction_mapping)
print("[GraphRAG Search]", "Found" if found else "Not Found", "|", description)

found, description = rq_rag_search(G, drug1_id, drug2_id, interaction_mapping)
print("[RQ-RAG Search]", "Found" if found else "Not Found", "|", description)


[Direct Edge Search] Found | #Drug1 may increase the anticoagulant activities of #Drug2.
[Neighbor Search] Found | #Drug1 may increase the anticoagulant activities of #Drug2.
[LightRAG Search] Found | Direct interaction: #Drug1 may increase the anticoagulant activities of #Drug2.
[GraphRAG Search] Found | Warfarin → #Drug1 may increase the anticoagulant activities of #Drug2. → Acetylsalicylic acid
[RQ-RAG Search] Found | Both drugs interact via Gemifloxacin.


In [4]:
list(G.nodes())[:50]


['DB08897',
 'DB00424',
 'DB00670',
 'DB06148',
 'DB01116',
 'DB00391',
 'DB00517',
 'DB09076',
 'DB01090',
 'DB01071',
 'DB01168',
 'DB00462',
 'DB00732',
 'DB01409',
 'DB00810',
 'DB00986',
 'DB00332',
 'DB00434',
 'DB00805',
 'DB00496',
 'DB06702',
 'DB00967',
 'DB00219',
 'DB00366',
 'DB01037',
 'DB01339',
 'DB00283',
 'DB01247',
 'DB00387',
 'DB01036',
 'DB00657',
 'DB01591',
 'DB00572',
 'DB00940',
 'DB09300',
 'DB00782',
 'DB00747',
 'DB00182',
 'DB00780',
 'DB01199',
 'DB01171',
 'DB01367',
 'DB00209',
 'DB00614',
 'DB00908',
 'DB00483',
 'DB00376',
 'DB00340',
 'DB01062',
 'DB09241']

In [5]:
for node in list(G.nodes())[:50]:
    print(node, ":", G.nodes[node]["name"])


DB08897 : Aclidinium
DB00424 : Hyoscyamine
DB00670 : Pirenzepine
DB06148 : Mianserin
DB01116 : Trimethaphan
DB00391 : Sulpiride
DB00517 : Anisotropine Methylbromide
DB09076 : Umeclidinium
DB01090 : Pentolinium
DB01071 : Mequitazine
DB01168 : Procarbazine
DB00462 : Methylscopolamine bromide
DB00732 : Atracurium besylate
DB01409 : Tiotropium
DB00810 : Biperiden
DB00986 : Glycopyrronium
DB00332 : Ipratropium bromide
DB00434 : Cyproheptadine
DB00805 : Minaprine
DB00496 : Darifenacin
DB06702 : Fesoterodine
DB00967 : Desloratadine
DB00219 : Oxyphenonium
DB00366 : Doxylamine
DB01037 : Selegiline
DB01339 : Vecuronium
DB00283 : Clemastine
DB01247 : Isocarboxazid
DB00387 : Procyclidine
DB01036 : Tolterodine
DB00657 : Mecamylamine
DB01591 : Solifenacin
DB00572 : Atropine
DB00940 : Methantheline
DB09300 : Butylscopolamine
DB00782 : Propantheline
DB00747 : Scopolamine
DB00182 : Amphetamine
DB00780 : Phenelzine
DB01199 : Tubocurarine
DB01171 : Moclobemide
DB01367 : Rasagiline
DB00209 : Trospium
DB00

In [6]:
from itertools import combinations

drug_ids = [
    "DB08897",  # Aclidinium
    "DB00424",  # Hyoscyamine
    "DB00670",  # Pirenzepine
    "DB06148",  # Mianserin
    "DB01116",  # Trimethaphan
    "DB00391",  # Sulpiride
    "DB00517",  # Anisotropine Methylbromide
    "DB09076",  # Umeclidinium
    "DB01090",  # Pentolinium
    "DB01071",  # Mequitazine
]

def search_drug_pair_ids(drug1_id, drug2_id):
    if drug1_id not in G.nodes or drug2_id not in G.nodes:
        return ("Node Error", "One or both drugs not in graph.")
    
    found, description = direct_edge_search(G, drug1_id, drug2_id, interaction_mapping)
    if found:
        return ("Direct Edge", description)

    found, description = neighbor_search(G, drug1_id, drug2_id, interaction_mapping)
    if found:
        return ("Neighbor", description)

    found, description = light_rag_search(G, drug1_id, drug2_id, interaction_mapping)
    if found:
        return ("LightRAG", description)

    found, description = adaptive_graph_rag_search(G, drug1_id, drug2_id, interaction_mapping)
    if found:
        return ("GraphRAG", description)

    found, description = rq_rag_search(G, drug1_id, drug2_id, interaction_mapping)
    if found:
        return ("RQ-RAG", description)

    return ("Not Found", "No interaction found by any method.")

results = []

for drug1_id, drug2_id in combinations(drug_ids, 2):
    if drug1_id not in G.nodes or drug2_id not in G.nodes:
        results.append((drug1_id, drug2_id, "Node Error", "One or both drugs not found"))
        continue

    drug1_name = G.nodes[drug1_id]['name']
    drug2_name = G.nodes[drug2_id]['name']

    method, desc = search_drug_pair_ids(drug1_id, drug2_id)
    results.append((drug1_name, drug2_name, method, desc))

print("{:<25} {:<25} {:<15} {}".format("Drug 1", "Drug 2", "Method", "Description"))
print("="*110)
for drug1, drug2, method, desc in results:
    print(f"{drug1:<25} {drug2:<25} {method:<15} {desc}")


Drug 1                    Drug 2                    Method          Description
Aclidinium                Hyoscyamine               Direct Edge     #Drug1 may increase the anticholinergic activities of #Drug2.
Aclidinium                Pirenzepine               Direct Edge     #Drug1 may increase the anticholinergic activities of #Drug2.
Aclidinium                Mianserin                 Direct Edge     #Drug1 may increase the anticholinergic activities of #Drug2.
Aclidinium                Trimethaphan              Direct Edge     #Drug1 may increase the anticholinergic activities of #Drug2.
Aclidinium                Sulpiride                 Direct Edge     #Drug1 may increase the anticholinergic activities of #Drug2.
Aclidinium                Anisotropine Methylbromide Direct Edge     #Drug1 may increase the anticholinergic activities of #Drug2.
Aclidinium                Umeclidinium              Direct Edge     #Drug1 may increase the anticholinergic activities of #Drug2.
Aclidiniu